In [1]:
import xarray as xr
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
f = xr.open_dataset('seNorge2018_RR_1957_2017.nc')

In [15]:
# f['time'][:]
f

<xarray.Dataset>
Dimensions:      (X: 1195, Y: 1550, nv: 2, time: 22280)
Coordinates:
  * X            (X) float64 -7.45e+04 -7.35e+04 ... 1.118e+06 1.12e+06
  * Y            (Y) float64 8e+06 7.998e+06 7.998e+06 ... 6.452e+06 6.450e+06
  * time         (time) datetime64[ns] 1957-01-01T06:00:00 ... 2017-12-31T06:00:00
    lon          (Y, X) float32 ...
    lat          (Y, X) float32 ...
  * nv           (nv) float64 0.0 1.0
Data variables:
    rr           (time, Y, X) float32 ...
    UTM_Zone_33  float64 ...
    time_bnds    (time, nv) datetime64[ns] ...
Attributes:
    institution:  Norwegian Meteorological Institute, MET Norway
    creator_url:  met.no
    license:      https://www.met.no/en/free-meteorological-data/Licensing-an...
    source:       MET Norway
    comment:      Our open data are licensed under Norwegian Licence for Open...
    Conventions:  CF-1.7
    title:        seNorge_2018 RR
    DOI:          10.5281/zenodo.2082320
    version:      18.12
    summary:      s

In [4]:
# round dates to 'day'
f_days = f.assign(date = lambda f: f.time.dt.round('D')) 
# add weekday
f_days_of_week = f_days.assign(day_of_week = lambda f: f_days.date.dt.weekday) 

In [9]:
f_days_of_week['day_of_week']

<xarray.DataArray 'day_of_week' (time: 22280)>
array([1, 2, 3, ..., 4, 5, 6])
Coordinates:
  * time     (time) datetime64[ns] 1957-01-01T06:00:00 ... 2017-12-31T06:00:00

In [22]:
# groups:
a = f_days_of_week.groupby("day_of_week").groups

# group by day of week
xmean = f_days_of_week.groupby('day_of_week').mean()

/opt/anaconda3/envs/rain/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [23]:
# what xmean looks like
xmean

<xarray.Dataset>
Dimensions:      (X: 1195, Y: 1550, day_of_week: 7, nv: 2)
Coordinates:
    lat          (Y, X) float32 71.394295 71.396675 ... 57.766502 57.765125
    lon          (Y, X) float32 -1.309146 -1.282192 ... 25.421093 25.437622
  * Y            (Y) float64 8e+06 7.998e+06 7.998e+06 ... 6.452e+06 6.450e+06
  * nv           (nv) float64 0.0 1.0
  * X            (X) float64 -7.45e+04 -7.35e+04 ... 1.118e+06 1.12e+06
  * day_of_week  (day_of_week) int64 0 1 2 3 4 5 6
Data variables:
    rr           (day_of_week, Y, X) float32 nan nan nan nan ... nan nan nan nan
    UTM_Zone_33  (day_of_week) float64 1.0 1.0 1.0 1.0 1.0 1.0 1.0

In [20]:
# define function to compute mean per group
def mean_rain(rr):
    return rr.mean()

In [21]:
# map() - apply function to groups: 
day_of_week_group = f_days_of_week.groupby("day_of_week")

day_of_week_group_mean = day_of_week_group.map(mean_rain)

AttributeError: 'DatasetGroupBy' object has no attribute 'map'

In [8]:
# f_days_of_week.day_of_week
